In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import sys
import os
print(os.getcwd())
sys.path.append("../")

from src.data_loader import get_combined_dataloaders
from src.models import *

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

c:\Users\aresf\Desktop\Code\HyperMask\notebooks
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
dataset_names = ["mnist", "omniglot", "fashion_mnist", "cifar10"]
train_loader, test_loader = get_combined_dataloaders(
    dataset_names, batch_size=128, equal_samples=True
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [9]:
x, y = next(iter(train_loader))

In [17]:
class GatedConvNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=6, kernel_size=5):
        super(GatedConvNet, self).__init__()
        self.hyper = HyperNetwork(in_channels, out_channels, kernel_size)
        self.primary = PrimaryNetwork(in_channels, out_channels, kernel_size)

    def apply_primary(self, sample, sample_mask):
        return self.primary(
            sample.unsqueeze(0), sample_mask.view_as(self.primary.conv.weight)
        )

    def forward(self, x):
        masks = self.hyper(x)
        return vmap(self.apply_primary)(x, masks).squeeze(1)

In [18]:
model = GatedConvNet()
output = model(x)
print("Output shape:", output.shape)

# Demonstrating that each sample in the batch gets a different mask
masks = model.hyper(x)
print("Unique mask values for first two samples:")
print("Are masks for first two samples identical?", torch.allclose(masks[0], masks[1]))

Output shape: torch.Size([128, 6, 24, 24])
Unique mask values for first two samples:
Are masks for first two samples identical? False


In [26]:
Ys = []
for x, y in train_loader:
    Ys.append(y.numpy())

In [21]:
y

tensor([  8,  14,  25, 673,   0,   7,   4,   3,   4,  41,   9,   0,   9,   5,
          0,   8,   5,   3,   5, 391,   9,   5, 162, 811,   2,   9, 599,   6,
          3, 690,   0,   5,   0,   9, 848,   9,  53, 508,   7,   2,   8,   1,
          6, 610,   3, 349,   9,   5,   0,   6,   3,   9,   0,   6,   4,   6,
        232, 341,   4,   9, 430,   5, 151,   7,   2,   0,   7,   0,   2,  95,
          1, 473,   9,   2,   0, 922,   2, 736,   7,   1,   8,  22,   8,   0,
        824,   6,   1,   6,   6,   3,   9,   9,   3,   8,   6,   9,   1,   9,
        249,   5,   3,   9,   8,   4,   7,   3,   4,   3,   6,   3,   9,   5,
          3, 925, 541,   4, 590,  87,   1,   9,   5, 527,   1,   9,   1, 642,
          0,   9])

In [7]:
masks = model.hypernetwork(x)

In [11]:
masks[0].shape

torch.Size([128, 150])

In [13]:
model.backbone.conv1.weight.shape

torch.Size([6, 1, 5, 5])

In [15]:
from functorch import vmap

In [ ]:
vmap

In [10]:
model = GatedLeNet()
input_tensor = torch.randn(32, 1, 28, 28)  # Batch of 32 grayscale images of size 28x28
output = model(input_tensor)

TypeError: super(type, obj): obj must be an instance or subtype of type

In [11]:
mask_conv1, mask_conv2, mask_fc1, mask_fc2, mask_fc3 = model.hypernetwork(x)

In [12]:
[x.shape for x in masks]

[torch.Size([128, 150]),
 torch.Size([128, 2400]),
 torch.Size([128, 48000]),
 torch.Size([128, 10080]),
 torch.Size([128, 840])]

In [14]:
mask_conv1.shape

torch.Size([128, 150])